In [ ]:
#### Packages installation
library(dplyr)
library(tidyverse)
library(ggplot2)

In [ ]:
### read the ICD codes data
icd <- read.delim("/home/ec2-user/studies/Research-Fellowship-jonathan-hernandez-agosto/tblICD10.csv", sep=",", header = FALSE)


In [ ]:
dim(icd)

In [ ]:
head(icd)

In [ ]:
#format the data
icd_formatted <- icd %>%
mutate( ICD = sapply(strsplit( as.character(V2), "[.]"), '[', 1), 
        ICD = gsub("ICD10CM:", "", ICD), 
        demog = sapply(strsplit( as.character(V3), "[:]"), '[', 2)) %>%
select( patient = V1, ICD, demog, year = V4 ) %>%
unique()

In [ ]:
head(icd_formatted)

In [ ]:
### estimate the frequency of each code per year
counts <- icd_formatted %>%
            dplyr::group_by( ICD, demog, year ) %>%
            dplyr::summarize( n = n())

head(counts)

In [ ]:
totals <- icd_formatted %>%
            dplyr::group_by( demog, year ) %>%
            dplyr::summarize( total = n_distinct(patient))

head(totals)

In [ ]:
frequencies <- counts %>%
                    left_join( totals , by = c("demog", "year")) %>%
                    mutate( perc = round( n/total*100, 2)) 

head(frequencies)

In [ ]:
#### visualization as heatmap of frequencies 
hispanic <- frequencies %>%
filter( demog == "Y")

non_hispanic <- frequencies %>%
filter( demog == "N")

In [ ]:
ggplot(hispanic, aes(year, ICD, fill= perc)) + 
  geom_tile() +
  geom_text(aes(label = perc)) +
  scale_fill_gradient(low="white", high="blue") +
  ggtitle("Hispanic: frequency of the codes over time") 

ggplot(non_hispanic, aes(year, ICD, fill= perc)) + 
  geom_tile() +
  geom_text(aes(label = perc)) +
  scale_fill_gradient(low="white", high="blue") +
  ggtitle("Non-hispanic: frequency of the codes over time") 


In [ ]:
#### same analysis but using as denominator total counts of patients with at least 1 ICD code
#### SQL code to get this total number of patients

#  SELECT A.PATIENT_NUM, C.HISPANIC_CD, year(A.START_DATE) AS YEAR_DIAG
#  INTO #tblTotalICDs
#  FROM FellowsSample.S11.observation_fact A
#  INNER JOIN FellowsSample.S11.PATIENT_DIMENSION C
#  On A.PATIENT_NUM = C.PATIENT_NUM
#  WHERE A.CONCEPT_CD like 'ICD10CM:%'
#  GROUP BY A.PATIENT_NUM, C.HISPANIC_CD, year(A.START_DATE);

#  select count( distinct( PATIENT_NUM)) as totalPatients, YEAR_DIAG, HISPANIC_CD 
#  from #tblTotalICDs
#  group by YEAR_DIAG, HISPANIC_CD; 

In [ ]:
totalCounts <- read.delim("totalPatientCounts_patientsWithAtleastOneICD.txt") %>%
                mutate( demog = sapply(strsplit( as.character(hispanic_cd), "[:]"), '[', 2)) %>%
                select( total = totalP, year, demog )

totalCounts

In [ ]:
frequencies_alt <- counts %>%
                    left_join( totalCounts , by = c("demog", "year")) %>%
                    mutate( perc = round( n/total*100, 3)) 

head(frequencies_alt)

In [ ]:
#### visualization as heatmap of frequencies 
hispanic_alt <- frequencies_alt %>%
filter( demog == "Y")

non_hispanic_alt <- frequencies_alt %>%
filter( demog == "N")

In [ ]:
ggplot(hispanic_alt, aes(year, ICD, fill= perc)) + 
  geom_tile() +
  geom_text(aes(label = perc)) +
  scale_fill_gradient(low="white", high="blue") +
  ggtitle("Hispanic: frequency of the codes over time with denominator updated") 

ggplot(non_hispanic_alt, aes(year, ICD, fill= perc)) + 
  geom_tile() +
  geom_text(aes(label = perc)) +
  scale_fill_gradient(low="white", high="blue") +
  ggtitle("Non-hispanic: frequency of the codes over time with denominator updated") 